In [1]:
%load_ext autoreload
%autoreload 2

## Init blockchain

In [2]:
from rpc import Proxy
import bitcoin

In [3]:
bitcoin.SelectParams('regtest')

In [4]:
rpc = Proxy()

In [5]:
rpc.generate(101)

<generator object Proxy.generate.<locals>.<genexpr> at 0x104caa308>

## Init Alice and Bob

In [6]:
from collections import namedtuple
import bitcoin.core as bc
import bitcoin.wallet as bw

In [9]:
class User(object):
    def __init__(self, rpc):
        self.rpc = rpc
        self.secrets = []
        self.addresses = []
    
    def generate_addresses(self, n):
        for _ in range(n):
            addr = self.rpc.getnewaddress()
            secr = self.rpc.dumpprivkey(addr)
            self.addresses.append(addr)
            self.secrets.append(secr)
    
    @property
    def utxos(self):
        txs = self.rpc.listunspent(addrs=self.addresses)
        return txs

In [10]:
ali = User(rpc)
bob = User(rpc)

ali.generate_addresses(5)
bob.generate_addresses(5)

In [14]:
_ = rpc.sendtoaddress(ali.addresses[0], 10 * bc.COIN)
_ = rpc.sendtoaddress(bob.addresses[0], 10 * bc.COIN)

In [16]:
rpc.generate(1)
curr_block = rpc.getinfo()['blocks']
curr_block

103

## Create $T^{Fund}$ scripts

In [18]:
import bitcoin.core.script as bs

In [19]:
cancel_time = curr_block + 4
fork_time = curr_block + 6
bob_lock_time = curr_block + 8
ali_lock_time = curr_block + 10

In [125]:
def get_refund_script(pk, delta):
    assert isinstance(pk, bc.key.CPubKey)
    assert isinstance(delta, int)
    assert delta > 0

    script = bs.CScript([
        delta,
        bs.OP_CHECKLOCKTIMEVERIFY,
        bs.OP_DROP,
        pk,
        bs.OP_CHECKSIG
    ])
    return script

def get_cancel_script(pk1, pk2):
    assert isinstance(pk1, bc.key.CPubKey)
    assert isinstance(pk2, bc.key.CPubKey)
    
    script = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])
    
    return script

In [123]:
ali_refund_script = get_refund_script(ali.secrets[1].pub, ali_lock_time)
bob_refund_script = get_refund_script(bob.secrets[1].pub, bob_lock_time)

In [126]:
cancel_script = get_cancel_script(ali.secrets[3].pub, bob.secrets[3].pub)

In [127]:
ali_deposit = bs.CScript([
    bs.OP_IF,
        *list(ali_refund_script),
    bs.OP_ELSE,
        *list(cancel_script),
    bs.OP_ENDIF])

In [128]:
bob_deposit = bs.CScript([
    bs.OP_IF,
        *list(bob_refund_script),
    bs.OP_ELSE,
        *list(cancel_script),
    bs.OP_ENDIF])

## Create $T^{Fund}$ transaction

In [154]:
ali_utxo = ali.utxos[0]
bob_utxo = bob.utxos[0]

In [155]:
ali_txin = bc.CMutableTxIn(ali_utxo['outpoint'], nSequence=0)
bob_txin = bc.CMutableTxIn(bob_utxo['outpoint'], nSequence=0)

In [156]:
fee = 15700.0  # 630 bytes

In [205]:
cancel_amount = 2

In [157]:
ali_deposit_txout = bc.CMutableTxOut(ali_utxo['amount'] - cancel_amount/2 - fee/2,
                                     ali_deposit)
bob_deposit_txout = bc.CMutableTxOut(bob_utxo['amount'] - cancel_amount/2 - fee/2,
                                     bob_deposit)

cancel_txout = bc.CMutableTxOut(cancel_amount, cancel_script)

In [158]:
tx_fund = bc.CMutableTransaction([ali_txin,
                                  bob_txin], [ali_deposit_txout,
                                              bob_deposit_txout, 
                                              cancel_txout])

In [159]:
ali_sighash = bs.SignatureHash(ali.addresses[0].to_scriptPubKey(),
                               tx_fund,
                               0,
                               bs.SIGHASH_ALL)

bob_sighash = bs.SignatureHash(bob.addresses[0].to_scriptPubKey(),
                               tx_fund,
                               1,
                               bs.SIGHASH_ALL)

In [160]:
ali_sig = ali.secrets[0].sign(ali_sighash) + bytes([bs.SIGHASH_ALL])
bob_sig = bob.secrets[0].sign(bob_sighash) + bytes([bs.SIGHASH_ALL])

In [161]:
ali_txin.scriptSig = bs.CScript([ali_sig, ali.secrets[0].pub])
bob_txin.scriptSig = bs.CScript([bob_sig, bob.secrets[0].pub])

In [162]:
# https://github.com/petertodd/python-bitcoinlib/blob/5e150ac4a50791e6293752ceef8647b9bb3273c0/examples/timestamp-op-ret.py#L66
FEE_PER_BYTE = 0.00025 * bc.COIN/1000

In [163]:
n_bytes = len(tx_fund.serialize())
fee = n_bytes * FEE_PER_BYTE
fee

15750.0

In [164]:
tx_fund_id = rpc.sendrawtransaction(tx_fund)

In [169]:
rpc.generate(1)

<generator object Proxy.generate.<locals>.<genexpr> at 0x105395468>

In [187]:
assert rpc.gettransaction(tx_fund_id)['confirmations'] == 1

## Create $T^{cancel}$

In [226]:
tx_fund = rpc.getrawtransaction(tx_fund_id)

In [227]:
ali_cancel_txin = bc.CMutableTxIn(bc.COutPoint(tx_fund_id, 0),
                                  nSequence=0)
bob_cancel_txin = bc.CMutableTxIn(bc.COutPoint(tx_fund_id, 1),
                                  nSequence=0)
can_cancel_txin = bc.CMutableTxIn(bc.COutPoint(tx_fund_id, 2),
                                  nSequence=0)

In [239]:
fee = 16050.0  # 638 bytes

In [240]:
assert cancel_amount == tx_fund.vout[2].nValue

In [241]:
# TODO change address id to 5
ali_cancel_txout = bc.CMutableTxOut(
    tx_fund.vout[0].nValue + cancel_amount/2 - fee/2,
    ali.addresses[4].to_scriptPubKey())
bob_cancel_txout = bc.CMutableTxOut(
    tx_fund.vout[1].nValue + cancel_amount/2 - fee/2,
    bob.addresses[4].to_scriptPubKey())

In [242]:
tx_cancel = bc.CMutableTransaction(
    [ali_cancel_txin, bob_cancel_txin, can_cancel_txin],
    [ali_cancel_txout, bob_cancel_txout])

In [243]:
ali_cancel_sighash = bs.SignatureHash(
    tx_fund.vout[0].scriptPubKey,
    tx_cancel,
    0,
    bs.SIGHASH_ALL)

bob_cancel_sighash = bs.SignatureHash(
    tx_fund.vout[1].scriptPubKey,
    tx_cancel,
    1,
    bs.SIGHASH_ALL)

can_cancel_sighash = bs.SignatureHash(
    tx_fund.vout[2].scriptPubKey,
    tx_cancel,
    2,
    bs.SIGHASH_ALL)

In [244]:
ali_cancel_ali_sig = ali.secrets[3].sign(ali_cancel_sighash) + bytes([bs.SIGHASH_ALL])
ali_cancel_bob_sig = bob.secrets[3].sign(ali_cancel_sighash) + bytes([bs.SIGHASH_ALL])

In [245]:
bob_cancel_ali_sig = ali.secrets[3].sign(bob_cancel_sighash) + bytes([bs.SIGHASH_ALL])
bob_cancel_bob_sig = bob.secrets[3].sign(bob_cancel_sighash) + bytes([bs.SIGHASH_ALL])

In [246]:
can_cancel_ali_sig = ali.secrets[3].sign(can_cancel_sighash) + bytes([bs.SIGHASH_ALL])
can_cancel_bob_sig = bob.secrets[3].sign(can_cancel_sighash) + bytes([bs.SIGHASH_ALL])

In [250]:
ali_cancel_txin.scriptSig = bs.CScript(
    [bs.OP_0, ali_cancel_ali_sig, ali_cancel_bob_sig, bs.OP_0])

bob_cancel_txin.scriptSig = bs.CScript(
    [bs.OP_0, bob_cancel_ali_sig, bob_cancel_bob_sig, bs.OP_0])

can_cancel_txin.scriptSig = bs.CScript(
    [bs.OP_0, can_cancel_ali_sig, can_cancel_bob_sig])

In [251]:
n_bytes = len(tx_cancel.serialize())
fee = n_bytes * FEE_PER_BYTE
fee

16075.0

In [252]:
tx_cancel_id = rpc.sendrawtransaction(tx_cancel)

In [253]:
rpc.generate(1)

<generator object Proxy.generate.<locals>.<genexpr> at 0x1052b8fc0>

In [254]:
assert rpc.gettransaction(tx_cancel_id)['confirmations'] == 1